# OpenStreetMap Data Case Study [SQL]
***
<P>
### Student: Daniel de Carvalho Rust
### Date: March 26th 2017
***

![Rio de Janeiro Image](rio.jpeg "Rio de Janeiro, Brazil")

## Map Region: Brazil, Rio de Janeiro Metropolitan Area

I've chosen to analyze an extract of Rio de Janeiro, Brazil (my hometown) using SQL.

The data was obtained from [Mapzen](https://s3.amazonaws.com/metro-extracts.mapzen.com/rio-de-janeiro_brazil.osm.bz2 "OSM File"). For code evaluation purposes, a sample file with every 10th element was created using the [sample.py](sample.py) code.

***
## 1. XML Structure
<p>
The XML structure is important to understand what to expect from the data we will analyze. The meaning of elements and its attributes, data type, etc. It makes easier to spot inconsistencies to be fixed and information that can be safely ignored.
***

### 1.1 Main Elements: **`<node>`**, **`<way>`** and **`<relation>`** 

According to OpenStreetMap Wiki ([link](http://wiki.openstreetmap.org/w/index.php?title=Elements&redirect=no)):

> Elements are the basic components of OpenStreetMap's conceptual data model of the physical world. They consist of
1. **`<node>`** (defining points in space) [wiki](http://wiki.openstreetmap.org/wiki/Node) , 
2. **`<way>`** (defining linear features and area boundaries) [wiki](http://wiki.openstreetmap.org/wiki/Way), and
3. **`<relation>`** (which are sometimes used to explain how other elements work together) [wiki](http://wiki.openstreetmap.org/wiki/Relation).

<p>
**A `<node>` is a single point in space** defined by its latitude, longitude and node id.
<p>
**A `<way>` is an ordered list of nodes**, which can be open or closed.<br> A closed way is one whose last node on the way is also the first on that way. A closed way may be interpreted either as a closed polyline, or an area, or both.
<p>
**A `<relation>` defines logical or geographic relationships between other elements**.


### 1.2  Main Elements Common Attributes

Each **`<node>`**, **`<way>`** or **`<relation>`** element may or may not have **attributes** associated to them. Again according to [wiki](http://wiki.openstreetmap.org/w/index.php?title=Elements&redirect=no#Node):

> ###### Common attributes <br>
> Within the OSM database, we store these attributes for nodes, ways and relations. Your application may not need to make use of all of them, and some third-party extracts produced from OSM data may not reproduce them all.
1. **`@id`**: Used for identifying the element. **Element types have their own ID space, so there could be a node with id=100 and a way with id=100, which are unlikely to be related or geographically near to each other.**
2. **`@user`**: The display name of the user who last modified the object (informative only and may be empty).
3. **`@uid`**: The numeric identifier of the user who last modified the object. An user identifier never changes.
4. **`@timestamp`**: W3C standard date and time formats.	Time of the last modification.
5. **`@visible`**: Whether the object is deleted or not in the database, if visible="false" then the object should only be returned by history calls.
6. **`@version`**: The edit version of the object. Newly created objects start at version 1 and the value is incremented by the server when a client uploads a new version of the object.
7. **`@changeset`**: The changeset number in which the object was created or updated                                                                                                                                                                         

### 1.3 Child Element: `<tag>`

**`<tag>`** elements are always child elements of nodes, ways and relations. <br>
Those, on the other hand, may or may not have nested `<tag>` elements.
<p>
`<tag>` elements have key/value attributes called **`@k`** and  **`@v`** respectively, describing characteristics about the parent element.<br>

***
## 2. Data Overview
<p>
The first step in our project is to analyze our file to check its structure and gather basic information. For example, which and how many tag types there are. In Openstreetmap case there's a dictionary available at their website, but that's not always the case. It's also necessary to check for unexpected tags which may need to be fixed.
***

#### 2.1 File Size

**Full datasets (not submitted):**<br>
rio-de-janeiro_brazil.osm --------------------- 353.2 MB<br>
rio_de_janeiro.db ------------------------------ 250.5 MB<p>

**Sample datasets:**<br>
rio-de-janeiro_brazil_sample.osm ------------- 35.6 MB<br>
rio_de_janeiro_sample.db ---------------------- 24.9 MB


#### 2.2 - Number of Unique Tags: 
Source code: [mapparser.py](mapparser.py)

- `<bounds>`: 1
- `<member>`: 25.291
- `<nd>`: 2.015.603
- `<node>`: 1.615.351
- `<osm>`: 1
- `<relation>`: 4.178
- `<tag>`: 627.853
- `<way>`: 193.769


#### 2.3 - Patterns: 
Source code: [tags.py](tags.py)

The `<tag>` tags contain key/value pairs named 'k' and 'v' respectively. 
The aim here was to explore the 'k' value for patterns and potential problems.
We used regular expressions to categorize the tags in four groups and count them.

- 'lower': 559.334 tags that contain only lowercase letters and are valid 
- 'lower_colon': 45.851 valid tags with a colon in their names
- 'problemchars': 1 tag with problematic characters
- 'other': 22.667 tags that do not fall into the other three categories


#### 2.4 - Number of Contributors:
Source code: [users.py](users.py) 

There are 1,396 unique contributing users in our database.

***
## 3. Encountered Problems
<p>
***

#### 3.1 - Street Type Inconsistencies:
The most recurring problem found in the dataset was the way street types were written. To overcome the problem we used regular expressions (regex) at the first element of the string, to match Brazilian street type descriptions. 

Source code: [audit.py](audit.py)
<p>
We have found two types of errors:

** 1 - Typos:** 
* Rue / Ruas => Rua
* Praca => Praça
<br>

** 2 - Abbreviations:** 
* R / R. => Rua
* Est / Estr => Estrada
* Av / Av. => Avenida
* Trav => Travessa
* Rod / Rod. => Rodovia
* Pca / Pça => Praça

#### 3.2 - ZIP Code Inconsistencies:
We modified the previous street type code to check for inconsistencies in ZIP codes.<p>

Brazilian ZIP code structure can be found [here](https://www.correios.com.br/para-voce/precisa-de-ajuda/o-que-e-cep-e-por-que-usa-lo/estrutura-do-cep). Although it's in Portuguese, the important aspects to keep in mind are:<br>
* The first digit must be '2' for Rio de Janeiro.
* The ZIP code must be 8 numbers long, with a dash after the 5th digit (2XXXX-XXX). 

The main problems found were:
* No minus sign '-', only integers. It was the most common problem found. It's understandable, many people write it that way.
* Odd characters, like backslashes, blank spaces and dots ('\', '.', ' ')
* Only 2 ZIP codes not starting with '2'
* ZIP codes with 5 numerical digits. The last 3 digits were added about 10 years ago and some sources still don't reflect the fact.

<p>
Source code: [audit_zipcodes.py](audit_zipcodes.py)<p>

***
## 4. XML to SQL Import
***

After auditing our data, we import them to our SQL database:

>**Step 1**: Fix the problems found while transforming the OSM file into tabular CSV files.
Source code: [data.py](data.py)

>**Step 2**: Import the CSV files to our SQL database.
Source code: [database.py](database.py)

We've chosen SQLite as our database manager for being light and independent of a client/server architecture.






***
## 5. Basic Descriptive Statistics and Sanity Checks
<p>
***

In [1]:
import sqlite3
import pprint

# Connecting to our database
db = sqlite3.connect("rio_de_janeiro.db")
cur = db.cursor()

### 5.1 Number of Nodes and Ways
<p>
**Do we have the same number of nodes and ways as in our original osm file?  **

In [2]:
nodes_count = cur.execute('SELECT count(*) FROM nodes')

print "number of nodes:", nodes_count.fetchone()[0]

number of nodes: 1615351


In [3]:
ways_count = cur.execute('SELECT count(*) FROM ways')

print "number of ways:", ways_count.fetchone()[0]

number of ways: 193769


Recapturing the results from 2.2 (Number of Unique Tags):

>{'bounds': 1,
 'member': 25291,
 'nd': 2015603,
 **'node': 1615351,**
 'osm': 1,
 'relation': 4178,
 'tag': 627853,
 **'way': 193769**}

**The import was sucessful. Both OSM and SQL files have the same number of nodes and ways.**

### 5.2 Correspondence between `ways_nodes` and `nodes` tables
<p>
Do all `node_id` in `ways_nodes` table have a correspondence in `nodes` table?
<p>
Note: 'OUTER JOIN' not supported in SQLite.

In [4]:
outer_join = cur.execute('''
SELECT
    ways_nodes.node_id AS missing_node_id,
    nodes.id AS null_id,
    count(*) AS qty
FROM 
    ways_nodes
    LEFT JOIN nodes ON ways_nodes.node_id = nodes.id
WHERE 
    null_id IS NULL
GROUP BY
    missing_node_id
''')

pprint.pprint(outer_join.fetchall())

[]


**OK! All `node_id` are filled with existing IDs in `nodes` table as expected.  **

### 5.3 Number of `tag` elements in `node` elements 

In [5]:
count_distinct = cur.execute('''
SELECT
    count(DISTINCT nodes_tags.id) AS distinct_qty
FROM 
    nodes_tags JOIN nodes ON nodes_tags.id = nodes.id
''')

pprint.pprint(count_distinct.fetchall())

[(52681,)]


**Total `node` elements**: 1,615,351<br>
**`node` elements with `tag` elements**: 52,681

**Only 3.2% of all `node` elements have `tag` elements**

### 5.4 Number of `tag` elements in `way` elements

In [6]:
count_distinct = cur.execute('''
SELECT
    count(DISTINCT ways_tags.id) AS distinct_qty
FROM 
    ways_tags JOIN ways ON ways_tags.id = ways.id
''')

pprint.pprint(count_distinct.fetchall())

[(189054,)]


**Total `way` elements**: 193,769<br>
**`way` elements with `tag` elements**: 189,054

**98% of all `way` elements have `tag` elements. <br>
It's a huge difference when compared to `node` elements!**

### 5.5 Validating `node` elements

In [7]:
node_key_types = cur.execute('''
SELECT
    type,
    count(*) AS qty
FROM 
    nodes_tags
GROUP BY
    type
ORDER BY
    qty DESC
LIMIT 10
''')

pprint.pprint(node_key_types.fetchall())

[(u'regular', 107791),
 (u'addr', 28232),
 (u'seamark', 715),
 (u'name', 494),
 (u'traffic_signals', 396),
 (u'payment', 250),
 (u'pt', 220),
 (u'tower', 139),
 (u'is_in', 116),
 (u'decreto', 96)]


** The types, with irrelevant exceptions, are ok for static elements like nodes.**<p>
** What about 'regular' type? What are the 'key' contents?**

In [8]:
regular_keys = cur.execute('''
SELECT
    type,
    key,
    count(*) AS qty
FROM 
    nodes_tags
WHERE
    type='regular'
GROUP BY
    type,
    key
ORDER BY
    qty DESC
LIMIT 10
''')

pprint.pprint(regular_keys.fetchall())

[(u'regular', u'name', 13018),
 (u'regular', u'source', 12268),
 (u'regular', u'highway', 11153),
 (u'regular', u'amenity', 10307),
 (u'regular', u'power', 5584),
 (u'regular', u'natural', 4564),
 (u'regular', u'shop', 3075),
 (u'regular', u'crossing', 2299),
 (u'regular', u'capacity', 1745),
 (u'regular', u'phone', 1667)]


** The 'regular' type keys show some unexpected values, like 'highway' for instance. Let's check 'highway' values.**

In [9]:
node_key_types = cur.execute('''
SELECT
    type,
    key,
    value,
    count(*) AS qty
FROM 
    nodes_tags
WHERE
    type='regular' and key='highway'
GROUP BY
    type,
    key,
    value
ORDER BY
    qty DESC
LIMIT 10
''')

pprint.pprint(node_key_types.fetchall())

[(u'regular', u'highway', u'traffic_signals', 4071),
 (u'regular', u'highway', u'crossing', 2935),
 (u'regular', u'highway', u'bus_stop', 1508),
 (u'regular', u'highway', u'turning_circle', 1375),
 (u'regular', u'highway', u'stop', 391),
 (u'regular', u'highway', u'speed_camera', 310),
 (u'regular', u'highway', u'mini_roundabout', 184),
 (u'regular', u'highway', u'street_lamp', 140),
 (u'regular', u'highway', u'motorway_junction', 100),
 (u'regular', u'highway', u'give_way', 82)]


** The 'highway' values are static indeed, so I consider node elements validated as a whole**. 

### 5.5 Validating `way` elements

In [10]:
way_key_types_regular = cur.execute('''
SELECT
    type,
    key,
    count(*) AS qty
FROM 
    ways_tags
WHERE
    type='regular'
GROUP BY
    type,
    key
ORDER BY
    qty DESC
LIMIT 10  
''')

pprint.pprint(way_key_types_regular.fetchall())

[(u'regular', u'highway', 113203),
 (u'regular', u'name', 55168),
 (u'regular', u'building', 52740),
 (u'regular', u'height', 33709),
 (u'regular', u'source', 33142),
 (u'regular', u'ele', 29514),
 (u'regular', u'oneway', 22422),
 (u'regular', u'top_ele', 17895),
 (u'regular', u'surface', 13761),
 (u'regular', u'layer', 6286)]


In [11]:
way_key_types_not_regular = cur.execute('''
SELECT
    type,
    key,
    count(*) AS qty
FROM 
    ways_tags
WHERE
    type!='regular'
GROUP BY
    type,
    key
ORDER BY
    qty DESC
LIMIT 10  
''')

pprint.pprint(way_key_types_not_regular.fetchall())

[(u'addr', u'street', 2167),
 (u'addr', u'interpolation', 1714),
 (u'addr', u'housenumber', 1629),
 (u'addr', u'inclusion', 1450),
 (u'addr', u'city', 1412),
 (u'building', u'levels', 1280),
 (u'addr', u'postcode', 1007),
 (u'addr', u'suburb', 747),
 (u'addr', u'country', 233),
 (u'turn', u'lanes', 212)]


**The most common 'way' keys are, as expected, related to "path" features: highway, height, oneway, lanes, bridge, access, maxspeed, etc.**
<p>
**But some very common ones are normally associated with static elements, which would probably fit better in nodes: building, housenumber, postcode, phone, religion, etc.**

***
## 6. Data Analysis
<p>
***

### 6.1 Top 10 Contributing Users

In [12]:
top_users = cur.execute('''
SELECT 
    joined_tables.user,
    COUNT(*) AS count
FROM 
    (
    SELECT user FROM nodes 
    UNION ALL
    SELECT user FROM ways
    )
    joined_tables
GROUP BY
    joined_tables.user
ORDER BY count DESC
LIMIT 10
''')

print "Top 10 Users:"
pprint.pprint(top_users.fetchall())

Top 10 Users:
[(u'Alexandrecw', 373474),
 (u'smaprs_import', 185068),
 (u'ThiagoPv', 184319),
 (u'AlNo', 163244),
 (u'Import Rio', 84927),
 (u'Geaquinto', 69218),
 (u'Nighto', 64816),
 (u'Ricardo Mitidieri', 58544),
 (u'Thundercel', 54138),
 (u'M\xe1rcio V\xedn\xedcius Pinheiro', 37055)]


### 6.2 Top 10 Amenities

In [13]:
top_ammenities = cur.execute('''
SELECT 
    value,
    COUNT(*) as count
FROM 
    nodes_tags
WHERE 
    key="amenity"
GROUP BY
    value
ORDER BY 
    count DESC
LIMIT 10
''')

pprint.pprint(top_ammenities.fetchall())

[(u'school', 1554),
 (u'bicycle_parking', 1475),
 (u'restaurant', 1040),
 (u'fast_food', 821),
 (u'bank', 498),
 (u'fuel', 455),
 (u'place_of_worship', 400),
 (u'pub', 391),
 (u'telephone', 374),
 (u'pharmacy', 353)]


### 6.3 Streets with Higher Concentration of Restaurants and Fast-Foods

In [14]:
restaurant_streets = cur.execute('''
SELECT 
    ways_tags.key,
    ways_tags.value,
    COUNT(*) AS qty_restaurants 
FROM 
    ways_nodes 
        JOIN ways_tags ON ways_tags.id = ways_nodes.id
        JOIN nodes_tags ON nodes_tags.id = ways_nodes.node_id
WHERE 
    ways_tags.key = "street" AND
    nodes_tags.key="amenity" AND
    (nodes_tags.value = 'restaurant' OR nodes_tags.value = 'fast_food')
GROUP BY
    ways_tags.key,
    ways_tags.value
ORDER BY qty_restaurants DESC
LIMIT 10
''')

pprint.pprint(restaurant_streets.fetchall())

[(u'street', u'Rua Da Concei\xe7\xe3o', 9),
 (u'street', u'Rua Do Catete', 4),
 (u'street', u'Rua Barata Ribeiro', 3),
 (u'street', u'Rua Das Laranjeiras', 2),
 (u'street', u'Avenida Prado Junior', 1),
 (u'street', u'Rua Da Quitanda', 1),
 (u'street', u'Rua Professor \xc1lvaro Ramos', 1),
 (u'street', u'Travessa Dos Tamoios', 1)]


** This result is very disapointing. There are 2167 streets, 1040 restaurants and 821 fast-foods in Rio's OpenStreet map.**<br>
Obviously there are many more streets with lots of restaurants, but they don't appear together with street names. 

### 6.4 Biggest Religions by Number of Temples

In [15]:
biggest_religion = cur.execute('''
SELECT 
    nodes_tags.value, 
    COUNT(*) as count
FROM 
    nodes_tags
        JOIN
        (
        SELECT
            DISTINCT(id)
        FROM
            nodes_tags
        WHERE 
            value="place_of_worship"
        ) i ON nodes_tags.id=i.id
WHERE
    nodes_tags.key="religion"
GROUP BY 
    nodes_tags.value
ORDER BY count DESC
LIMIT 3
''')

pprint.pprint(biggest_religion.fetchall())

[(u'christian', 343), (u'spiritualist', 6), (u'jewish', 5)]


#### 6.6 Popular Cuisines

In [16]:
popular_cuisines = cur.execute('''
SELECT 
    nodes_tags.value, 
    COUNT(*) as count
FROM 
    nodes_tags
        JOIN
        (
        SELECT
            DISTINCT(id)
        FROM
            nodes_tags
        WHERE 
            value="restaurant"
        ) i ON nodes_tags.id=i.id
WHERE
    nodes_tags.key="cuisine"
GROUP BY 
    nodes_tags.value
ORDER BY count DESC
LIMIT 10
''')


pprint.pprint(popular_cuisines.fetchall())

[(u'pizza', 82),
 (u'regional', 75),
 (u'italian', 38),
 (u'japanese', 36),
 (u'brazilian', 20),
 (u'steak_house', 16),
 (u'barbecue', 13),
 (u'international', 11),
 (u'chinese', 8),
 (u'seafood', 8)]


***
## 7. Conclusion
***

Data about Rio de Janeiro encompasses a larger area beyond its political limits, known as "Rio de Janeiro Metropolitan Area".<p>

The existing data is of a fair quality for an open source and often manually inputted data, although some 'static' elements, which should fit well in `<node>` elements, were found in `<way>` elements. To a reasonable extent the work done in this project to clean the data was well performed, but there should be a way to prevent users from inserting the most common typos. <p>

A particular problem found during the wrangling phase could not be fixed: the lack of information about `<node>` elements, with only 3% of them containing `<tag>` elements. For a successful fix, external data with lat/long coordinates should be used to feed our map. Unfortunately, there isn't such database freely available at the present time.<p>

The lack of information on `<node>` elements could be associated to the fact that many of them only exist as reference to `<way>` elements, indicating the orderly "dots" (static locations) along a path.
But this wouldn't explain the lack of information about restaurants in a city with 6 Million inhabitants. The largest concentration points to only 9 restaurants on the same street, while the second position has only 4 of them. There is clearly a problem of poor POI (points of interest) coverage. 

### Suggestions and Ideas

The opportunities to improve the map are endless, below are a few of them:

###### 1 - Facilitate user collaboration
It should be simple and straightforward for users to collaborate on the go, making it a live interaction. With thousands or millions of users reporting outdated, missing or misplaced information as they use the map, it can be up to date more often than not. The more the project depends on "super users" to upload information, the greatest are the chances of the map being misleading. The more accurate, the more users it will attract, creating a virtuous cycle.   

###### 2 - GPS Data from Public or Partner Sources  
Batch importing public GPS information into the map would significantly improve it, bringing more people to visit and contribute to it. Another possibility is to find private partners willing to further develop the solution by providing data in exchange for some sort of advanced privileges or rights. Examples of such sources are parcel services and government bodies related to transport, business or tourism sectors.  

###### 3 - Predefined Values
To avoid typos, lowercase/uppercase, language and other problems, the 'k' tags could be globally curated and get it's own id. Those not in the list (which could also not be fixed into one of the predefined categories by wrangling) would go into an 'other' category. This way tag names would be standardized from the start, not allowing users to freely input whatever name they wish. Those contributors willing to improve the map would focus on the 'other' category or on new data.

###### 4 - Language Synonyms
Language synonyms could be placed in another tag to be created, using a new 'id' reference. This is easy after standardizing the 'k' tag values. This way, tourists would be able to find landmarks and ways using their mother language. 

###### 5 - Automated cleaning of existing records
There should be an effort to batch clean existing map data using a bot, while also avoiding the input of typos and other errors by users (for example by the use of predefined values, as explained above). 

<p>
With these simple measures, OpenStreetMap could better compete with solutions offered by Google (Google Maps) and Microsoft (Bing), which are at the present more complete and user friendly.